In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [20]:
stfold = StratifiedKFold(n_splits=5, shuffle=True)
train_l = list()
val_l = list()
test_l = list()
maxlen = 100
cnt = 0
for train_index, test_index in stfold.split(data['comment'], data['label']):
    cnt += 1
    train = data.iloc[train_index]
    train_x, train_y = train['comment'], train['label']
    tmp = data.iloc[test_index]

    val, test = train_test_split(tmp, test_size=0.5, stratify=tmp['label'])    
    val_x, val_y = val['comment'], val['label']
    test_x, test_y = test['comment'], test['label']    
    
    train_x = vectorizer(np.array([[s] for s in train_x])).numpy()    
    val_x = vectorizer(np.array([[s] for s in val_x])).numpy()
    test_x = vectorizer(np.array([[s] for s in test_x])).numpy()

    train_x = keras.preprocessing.sequence.pad_sequences(train_x, maxlen=maxlen)
    val_x = keras.preprocessing.sequence.pad_sequences(val_x, maxlen=maxlen)
    test_x = keras.preprocessing.sequence.pad_sequences(test_x, maxlen=maxlen)
    

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = layers.Embedding(input_dim = vocab_size, 
                                       output_dim = emb_dim, 
                                       embeddings_initializer=keras.initializers.Constant(matrix),
                                       trainable=True)
    x = embedding_layer(inputs)
    x = layers.Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = layers.GlobalAveragePooling1D()(x)
    max_pool = layers.GlobalMaxPooling1D()(x)
    x = layers.concatenate([avg_pool, max_pool])
    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    model_path = "logistic_models/{itr:04}".format(itr=cnt)
    checkpoint_path = model_path + "-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    !mkdir logistic_models
    !mkdir $model_path
    
    model.compile(tf.optimizers.Adam(lr=0.001), "binary_crossentropy", metrics=[  "accuracy", 
                                                                                  tf.keras.metrics.AUC(name='auc')])
    
    
    #create checkpoint to save model
    #with best validation loss
    model.save_weights(checkpoint_path.format(epoch=0))

    checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1,
            save_weights_only=True, save_best_only=True, mode='auto')
    
    history = model.fit(
                        train_x, train_y, 
                        batch_size=128, epochs=10, 
                        validation_data=(val_x, val_y),
                        callbacks = [checkpoint]
                        )
    
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest)
    
    train_eval = model.evaluate(train_x, train_y)
    val_eval = model.evaluate(val_x, val_y)
    test_eval = model.evaluate(test_x, test_y)
    
    test_l.append(test_eval)
    val_l.append(val_eval)
    train_l.append(train_eval)

Epoch 1/10
63/63 [==============================] - ETA: 0s - loss: 0.6677 - accuracy: 0.5878 - auc: 0.6390
Epoch 00001: val_loss improved from inf to 0.62191, saving model to logistic_models/0001-0001.ckpt
63/63 [==============================] - 13s 201ms/step - loss: 0.6677 - accuracy: 0.5878 - auc: 0.6390 - val_loss: 0.6219 - val_accuracy: 0.7110 - val_auc: 0.7821
Epoch 2/10
63/63 [==============================] - ETA: 0s - loss: 0.3919 - accuracy: 0.8736 - auc: 0.9575
Epoch 00002: val_loss improved from 0.62191 to 0.33831, saving model to logistic_models/0001-0002.ckpt
63/63 [==============================] - 12s 193ms/step - loss: 0.3919 - accuracy: 0.8736 - auc: 0.9575 - val_loss: 0.3383 - val_accuracy: 0.8690 - val_auc: 0.9283
Epoch 3/10
63/63 [==============================] - ETA: 0s - loss: 0.0903 - accuracy: 0.9784 - auc: 0.9959
Epoch 00003: val_loss improved from 0.33831 to 0.30149, saving model to logistic_models/0001-0003.ckpt
63/63 [==============================] - 12

In [21]:
test_l = np.array(test_l)
val_l = np.array(val_l)
train_l = np.array(train_l)

print("test avg loss: ", np.mean(test_l[:, 0]), "+/-" ,np.std(test_l[:, 0]))
print("test avg acc: ", np.mean(test_l[:, 1]),  "+/-" ,np.std(test_l[:, 1]))
print("test avg auc: ", np.mean(test_l[:, 2]),  "+/-" ,np.std(test_l[:, 2]))
print('\n')


print("val avg loss: ", np.mean(val_l[:, 0]), "+/-" ,np.std(val_l[:, 0]))
print("val avg acc: ", np.mean(val_l[:, 1]),  "+/-" ,np.std(val_l[:, 1]))
print("val avg auc: ", np.mean(val_l[:, 2]),  "+/-" ,np.std(val_l[:, 2]))

print('\n')


print("train avg loss: ", np.mean(train_l[:, 0]), "+/-" ,np.std(train_l[:, 0]))
print("train avg acc: ", np.mean(train_l[:, 1]),  "+/-" ,np.std(train_l[:, 1]))
print("train avg auc: ", np.mean(train_l[:, 2]),  "+/-" ,np.std(train_l[:, 2]))

test avg loss:  0.3139005541801453 +/- 0.019148796034994284
test avg acc:  0.876800000667572 +/- 0.005635588639548361
test avg auc:  0.9358309864997864 +/- 0.007488603459061958


val avg loss:  0.3101997137069702 +/- 0.01650551720961471
val avg acc:  0.8824000000953675 +/- 0.008380944875579997
val avg auc:  0.9358522415161132 +/- 0.00825527811235876


train avg loss:  0.025912758708000184 +/- 0.003996942221474518
train avg acc:  0.9966249942779541 +/- 0.0004808931292703015
train avg auc:  0.9994903802871704 +/- 7.165988589970545e-05
